In [ ]:
!pip install ipython-autotime
%load_ext autotime

In [ ]:
!df -h

### 데이터 다운로드

In [ ]:
!ls -al

In [ ]:
!rm -rf ./datasets

In [ ]:
!ls -al

In [ ]:
import os
import tarfile
from six.moves import urllib

DATA_DIR = "./datasets"

# 디렉토리 만들기
if not os.path.isdir(DATA_DIR):
    os.makedirs(DATA_DIR)

In [ ]:
!ls -al

In [ ]:
!ls -al ./datasets/

In [ ]:
# housing.tgz 다운로드
DATA_PATH = os.path.join(DATA_DIR, "housing.tgz") # ./datasets/housing.tgz
urllib.request.urlretrieve(
    "https://raw.githubusercontent.com/ageron/handson-ml/master/datasets/housing/housing.tgz", 
    DATA_PATH)

In [ ]:
!ls -al ./datasets/

In [ ]:
# 압축 풀기
tgz_file = tarfile.open(DATA_PATH)
tgz_file.extractall(path=DATA_DIR)
tgz_file.close()


In [ ]:
!ls -al ./datasets/

#### 데이터 구조 살펴보기

In [ ]:
# CSV 파일 읽기
import pandas as pd

def load_data(file_dir=DATA_DIR):
    csv_file = os.path.join(file_dir, "housing.csv")
    return pd.read_csv(csv_file)
    
housing = load_data()


In [ ]:
type(housing)

In [ ]:
housing.head()

In [ ]:
!head -5 ./datasets/housing.csv

In [ ]:
!tail -5 ./datasets/housing.csv

In [ ]:
housing.info()

In [ ]:
housing["ocean_proximity"].value_counts()

In [ ]:
housing.describe()

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
housing.hist(bins=50, figsize=(20,15))
plt.show()


#### 테스트 데이터 셋 만들기

In [ ]:
import numpy as np

def split_train_test(data, test_ratio):
    shuffled_indices = np.random.permutation(len(data))
    test_set_size = int(len(data) * test_ratio)
    test_indices = shuffled_indices[:test_set_size]
    train_indices = shuffled_indices[test_set_size:]
    return data.iloc[train_indices], data.iloc[test_indices]

# train : test = 80 : 20
train_set, test_set = split_train_test(housing, 0.2)
print("train: {}, test: {}".format(len(train_set), len(test_set)))

In [ ]:
from sklearn.model_selection import train_test_split

# train : test = 80 : 20
train_set, test_set = train_test_split(housing, 
                                       test_size=0.2, 
                                       random_state=42)
print("train: {}, test: {}".format(len(train_set), len(test_set)))

In [ ]:
# 소득의 카테고리 만들기
# 1.5 : 소득 카테고리 수를 제한하기 위한 값 (1~10)
# set(housing['median_income'].astype(int))
housing["income_cat"] = np.ceil(housing["median_income"] / 1.5)
# 5 이상인 값은 5로 합침
housing["income_cat"].where(housing["income_cat"] < 5, 5.0, inplace=True)
# 히스토그램
housing["income_cat"].hist()

In [ ]:
housing["income_cat"].describe()

#### 계층 샘플링

In [ ]:
from sklearn.model_selection import StratifiedShuffleSplit

split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
for train_index, test_index in split.split(housing, housing["income_cat"]):
    strat_train_set = housing.loc[train_index]
    strat_test_set = housing.loc[test_index]
    
# 소득 카테고리 비율 살펴보기
housing["income_cat"].value_counts() / len(housing)

In [ ]:
strat_train_set["income_cat"].hist()

In [ ]:
strat_test_set["income_cat"].hist()

In [ ]:
housing["income_cat"].value_counts() / len(housing)

In [ ]:
strat_test_set["income_cat"].value_counts() / len(strat_test_set)

### 탐색과 시각화

#### 지리적 데이터 시각화

In [ ]:
# 훈련 데이터 복사
housing = strat_train_set.copy()

# 위도, 경도에 따른 산점도 그리기
housing.plot(kind="scatter", x="longitude", y="latitude")

In [ ]:
housing.plot(kind="scatter", x="longitude", y="latitude", alpha=0.1)

In [ ]:
housing.plot(kind="scatter", x="longitude", y="latitude", alpha=0.4,
            s=housing["population"]/100, label="population", figsize=(10,7),
            c="median_house_value",
            cmap=plt.get_cmap("jet"),
            colorbar=True, sharex=False)
plt.legend()

In [ ]:
type(housing)

In [ ]:
housing.corr()

In [ ]:
corr_matrix = housing.corr()
corr_matrix["median_house_value"].sort_values(ascending=False)

In [ ]:
from pandas.plotting import scatter_matrix

attributes = ["median_house_value", "median_income",
              "total_rooms", "housing_median_age"]
scatter_matrix(housing[attributes], figsize=(12,8))

In [ ]:
housing.plot(kind="scatter", x="median_income", 
             y="median_house_value", alpha=0.1)

#### 특성들의 조합

In [ ]:
housing["rooms_per_household"] = \
    housing["total_rooms"]/housing["households"]
housing["bedrooms_per_room"] = \
    housing["total_bedrooms"]/housing["total_rooms"]
housing["population_per_household"] = \
    housing["population"]/housing["households"]

In [ ]:
corr_matrix = housing.corr()
corr_matrix["median_house_value"].sort_values(ascending=False)

## 알고리즘용 데이터 준비

### 데이터 정제

In [ ]:
# 훈련 데이터 복사
housing = strat_train_set.drop("median_house_value", axis=1)
housing_labels = strat_train_set["median_house_value"].copy()
housing.count()

In [ ]:
# 해당 구역을 제거
housing2 = housing.dropna(subset=["total_bedrooms"])
housing2.count()

In [ ]:
# 특정 값으로 채움
median = housing["total_bedrooms"].median()
housing["total_bedrooms"].fillna(median, inplace=True)
housing.count()

In [ ]:
# 전체 특성을 삭제
housing2 = housing.drop("total_bedrooms", axis=1)
housing2.count()

In [ ]:
!pip show scikit-learn

In [ ]:
#from sklearn.preprocessing import Imputer
from sklearn.impute import SimpleImputer

# 중간값으로 대체하는 객체 생성
# mean, median, most_frequent, constant
#imputer = Imputer(strategy="median")
imputer = SimpleImputer(strategy="median")

# 수치형 데이터만 선택
housing_num = housing.drop("ocean_proximity", axis=1)
# 학습 및 변환
imputer.fit(housing_num)
X = imputer.transform(housing_num)


In [ ]:
# pandas DataFrame 으로 변환
housing_df = pd.DataFrame(X, columns=housing_num.columns,
                         index=list(housing.index.values))
housing_df.count()

In [ ]:
housing.index

In [ ]:
housing.info()


In [ ]:
housing.head()

#### 텍스트와 범주형 데이터

In [ ]:
housing_cat = housing["ocean_proximity"]
housing_cat.head(10)

In [ ]:
housing_cat.factorize()

In [ ]:
type(housing_cat)

In [ ]:
housing_cat_encoded, housing_categories = housing_cat.factorize()
housing_cat_encoded[:10]

In [ ]:
housing_categories

In [ ]:
from sklearn.preprocessing import OneHotEncoder
encoder = OneHotEncoder()
# fit_transform()에는 2차원 배열을 넣어야 함
housing_cat_1hot = encoder.fit_transform(
                housing_cat_encoded.reshape(-1,1))
# 출력은 sparse matrix (scipy.sparse.csr.csr_matrix)
housing_cat_1hot

In [ ]:
type(housing_cat_1hot)

In [ ]:
# shape : (16354, 5)
housing_cat_1hot.toarray()

In [ ]:
housing_cat_1hot.toarray().shape

In [ ]:
# OrdinalEncoder

from sklearn.preprocessing import OrdinalEncoder
# 2차원(DataFrame) : reshape(-1,1)과 같은 효과
housing_cat_2d = housing[["ocean_proximity"]]
ordinal_encoder = OrdinalEncoder()
housing_cat_encoded = \
            ordinal_encoder.fit_transform(housing_cat_2d)
housing_cat_encoded[:10]

In [ ]:
type(housing_cat_2d)

In [ ]:
housing_cat_2d.shape

In [ ]:
ordinal_encoder.categories_

In [ ]:
# 2차원(DataFrame) : reshape(-1,1)과 같은 효과
housing_cat_2d = housing[["ocean_proximity"]]

# default : sparse=True
cat_encoder = OneHotEncoder(sparse=False)
housing_cat_1hot = cat_encoder.fit_transform(housing_cat_2d)
housing_cat_1hot

In [ ]:
# 2차원(DataFrame) : reshape(-1,1)과 같은 효과
housing_cat_2d = housing_cat.values.reshape(-1,1)

# default : sparse=True
cat_encoder = OneHotEncoder(sparse=False)
housing_cat_1hot = cat_encoder.fit_transform(housing_cat_2d)
housing_cat_1hot

In [ ]:
housing_cat

In [ ]:
housing.describe()


In [ ]:
housing.iloc[:5]

In [ ]:
housing_labels

In [ ]:
housing_num=housing.drop("ocean_proximity",axis=1)
#imputer = Imputer(strategy="median")
imputer = SimpleImputer(strategy="median")
imputer.fit(housing_num)
X1 = imputer.transform(housing_num)


In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X1)
X2 = scaler.transform(X1)


In [ ]:
X2


In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

num_pipeline = Pipeline([
#    ('imputer', Imputer(strategy="median")),
    ('imputer', SimpleImputer(strategy="median")),
    ('scaler', StandardScaler()),
])
housing_num_tr = num_pipeline.fit_transform(housing_num)

In [ ]:
housing_num_tr

In [ ]:
from sklearn.compose import ColumnTransformer

# list 타입
num_attribs = list(housing_num)
cat_attribs = ["ocean_proximity"]

full_pipeline = ColumnTransformer([
        ("num", num_pipeline, num_attribs),
        ("cat", OneHotEncoder(), cat_attribs),
    ])

housing_prepared = full_pipeline.fit_transform(housing)
housing_prepared.shape

In [ ]:
housing.info()

In [ ]:
housing_prepared.shape

## 모델 선택과 모델 학습

In [ ]:
# 선형 회귀 모델 
from sklearn.linear_model import LinearRegression
lin_reg = LinearRegression()
# 학습
lin_reg.fit(housing_prepared, housing_labels)
# 예측
sample_data = housing.iloc[:5]
sample_labels = housing_labels.iloc[:5]
sample_data_prepared = full_pipeline.transform(sample_data)
print("predict:\n{}".format(lin_reg.predict(sample_data_prepared)))
print("target:\n{}".format(list(sample_labels)))

In [ ]:
from sklearn.metrics import mean_squared_error
housing_predict = lin_reg.predict(housing_prepared)
lin_mse = mean_squared_error(housing_labels, 
                             housing_predict)
lin_rmse = np.sqrt(lin_mse)
lin_rmse

In [ ]:
# Decision Tree
from sklearn.tree import DecisionTreeRegressor
# 모델 선택
tree_reg = DecisionTreeRegressor()
# 모델 학습
tree_reg.fit(housing_prepared, housing_labels)
# 예측
housing_predict = tree_reg.predict(housing_prepared)
# 평가
tree_mse = mean_squared_error(housing_labels, 
                             housing_predict)
tree_rmse = np.sqrt(tree_mse)
tree_rmse

#### 교차 검증

In [ ]:
from sklearn.model_selection import cross_val_score
# Linear regression
scores = cross_val_score(lin_reg, 
                         housing_prepared,
                         housing_labels, 
                         scoring="neg_mean_squared_error", 
                         cv=10)
scores_rmse = np.sqrt(-scores)
print("LinearRegression score:", scores_rmse.mean())
# Decisoin tree
scores = cross_val_score(tree_reg, 
                         housing_prepared,
                         housing_labels, 
                         scoring="neg_mean_squared_error", 
                         cv=10)
scores_rmse = np.sqrt(-scores)
print("DecisionTree score", scores_rmse.mean())

In [ ]:
# Random Forest
from sklearn.ensemble import RandomForestRegressor
forest_reg = RandomForestRegressor()
forest_reg.fit(housing_prepared, housing_labels)
scores = cross_val_score(forest_reg, 
                         housing_prepared,
                         housing_labels, 
                         scoring="neg_mean_squared_error", 
                         cv=10)
scores_rmse = np.sqrt(-scores)
print("RandomForest score", scores_rmse.mean())

In [ ]:
housing_predict = forest_reg.predict(housing_prepared)
np.sqrt(mean_squared_error(housing_labels, housing_predict))

#### Grid Search

In [ ]:
from sklearn.model_selection import GridSearchCV
# 파라미터 조합
param_grid = [
    {'n_estimators': [3,10,30], 'max_features': [2,4,6,8]},
    {'bootstrap': [False], 'n_estimators': [3, 10],
                                'max_features': [2,3,4]}
]
forest_reg = RandomForestRegressor()
# grid search
grid_search = GridSearchCV(forest_reg, param_grid, cv=5,
                          scoring='neg_mean_squared_error',
                          return_train_score=True)
grid_search.fit(housing_prepared, housing_labels)
grid_search.best_params_

In [ ]:
# 최적값 확인
grid_search.best_estimator_

In [ ]:
# score 확인
cvres = grid_search.cv_results_
for mean_score, params in zip(cvres["mean_test_score"],
                             cvres["params"]):
    print(np.sqrt(-mean_score), params)

#### 최종 모델 평가

In [ ]:
# 최종 모델 선택
final_model = grid_search.best_estimator_

# test dataset 으로 평가
X_test = strat_test_set.drop("median_house_value", axis=1)
y_test = strat_test_set["median_house_value"].copy()

# 전처리
X_test_prepared = full_pipeline.transform(X_test)
# 최종 예측
final_predict = final_model.predict(X_test_prepared)
# 평가
final_mse = mean_squared_error(y_test, final_predict)
final_rmse = np.sqrt(final_mse)
final_rmse

In [ ]:
!ls -al

## 제출
